# Investment Comp Screener

Premia/ Discount (P/D)

Sector 3 year Z Score (S3ZS)

Company 3 year Z Score (C3ZS)

In [1]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm
import os
import glob
import re
import seaborn as sns

from statsmodels.distributions.empirical_distribution import ECDF

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def calc_ecdf(data):
    #low is good
    ecdf= ECDF(data)
    current_perc = ecdf(data[-1])
    
    return(current_perc)

In [3]:
def color_negative_red(value):
    """
      Colors elements in a dateframe
      green if positive and red if
      negative. Does not color NaN
      values.
      """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

In [4]:
def Z_score(data):
    # not including the current observation
    std = data.iloc[:-1].std()
    mean = data.iloc[:-1].mean()
    Z = (data[-1:] - mean)/std
    return Z
    

In [5]:
def mod_Z_score(data):
    mean = np.mean(data.iloc[:-1])
    deviation_from_med = np.array(data.iloc[:-1]) - mean
    MAD = np.mean(np.abs(deviation_from_med))
    mod_Z = (data[-1:] - mean)/(MAD * 1.4826)
    return mod_Z

In [6]:
def signal(rank):
    if rank <= 2: return 1
    if rank <= 4: return 0.5
    if rank <= 7: return 0
    if rank <= 9: return -0.5
    if rank > 9: return -1

## Data Import

In [7]:
path = "holding_companies_data"
arr = [x for x in os.listdir(path) if x.endswith(".csv")]

In [8]:
for f in arr:
    print(f)

2022-05-19_bure_pris_historik.csv
2022-05-19_indu c_pris_historik.csv
2022-05-19_svol b_pris_historik.csv
2022-05-19_lund b_pris_historik.csv
2022-05-19_trac b_pris_historik.csv
2022-05-19_ores_pris_historik.csv
2022-05-19_kinv b_pris_historik.csv
2022-05-19_lato b_pris_historik.csv
2022-05-19_cred a_pris_historik.csv
2022-05-19_vnv_pris_historik.csv
2022-05-19_inve b_pris_historik.csv


In [9]:
for k in arr:
    if "trac" in k:
        traction = pd.read_csv(os.path.join(path,k))
    if "lund" in k:
        lundbergs = pd.read_csv(os.path.join(path,k))
    if "hav" in k:
        havsfrun = pd.read_csv(os.path.join(path,k))
    if "ores" in k:
        oresund = pd.read_csv(os.path.join(path,k))
    if "indu" in k:
        industri = pd.read_csv(os.path.join(path,k))
    if "svol" in k:
        svolder = pd.read_csv(os.path.join(path,k))
    if "bure" in k:
        bure = pd.read_csv(os.path.join(path,k))
    if "cred" in k:
        creades = pd.read_csv(os.path.join(path,k))
    if "inve" in k:
        investor = pd.read_csv(os.path.join(path,k))
    if "kinv" in k:
        kinnevik = pd.read_csv(os.path.join(path,k))
    if "vnv" in k:
        vnv = pd.read_csv(os.path.join(path,k))
    if "naxs" in k:
        naxs = pd.read_csv(os.path.join(path,k))
    if "lato" in k:
        latour = pd.read_csv(os.path.join(path,k))
    

## Calculate Metrics

In [10]:
look_back = 4*252 #3 years
ST_look_back = 5

companies = [traction, lundbergs, oresund, industri,
             svolder, bure, creades, investor, kinnevik, vnv,latour]

companies_names = ['Traction', 'Lundbergsföretagen', 'Öresund', 'Industrivärden',
             'Svolder', 'Bure', 'Creades', 'Investor', 'Kinnevik', 'VNV Global','Latour']

for comp in companies:
    
    comp['DATUM'] = pd.to_datetime(comp['DATUM'])
    comp.set_index(['DATUM'], inplace=True, drop=True)
    comp['P/D']=comp['PRIS']/comp['BERÄKNAT_SUBSTANSVÄRDE'] - 1

In [11]:
for comp in companies:
    comp['ADJ PRIS'] = comp['PRIS']
    comp['ADJ NAV'] = comp['BERÄKNAT_SUBSTANSVÄRDE']

 #KINNEVIK ZALANDO 
ZALANDO_DATE= companies[8].index.searchsorted(datetime(2021, 5, 17))
ZALANDO_AMOUNT= 864.60 * (28/143)
companies[8]['ADJ PRIS'].iloc[ZALANDO_DATE:] = companies[8]['ADJ PRIS'].iloc[ZALANDO_DATE:] + ZALANDO_AMOUNT
companies[8]['ADJ NAV'].iloc[ZALANDO_DATE:] = companies[8]['ADJ NAV'].iloc[ZALANDO_DATE:] + ZALANDO_AMOUNT  


## CREADES SPLIT (6)
SPLIT_DATE = companies[6].index.searchsorted(datetime(2021, 4, 27))
SPLIT_FACTOR = 10
companies[6]['ADJ PRIS'].iloc[SPLIT_DATE:] = companies[6]['ADJ PRIS'].iloc[SPLIT_DATE:] * SPLIT_FACTOR
companies[6]['ADJ NAV'].iloc[SPLIT_DATE:] = companies[6]['ADJ NAV'].iloc[SPLIT_DATE:] * SPLIT_FACTOR


### INVESTOR SPLIT (7)

SPLIT_DATE_INVESTOR = companies[7].index.searchsorted(datetime(2021, 5, 19))
SPLIT_FACTOR_INVESTOR = 4
companies[7]['ADJ PRIS'].iloc[SPLIT_DATE_INVESTOR:] = companies[7]['ADJ PRIS'].iloc[SPLIT_DATE_INVESTOR:] * SPLIT_FACTOR_INVESTOR
companies[7]['ADJ NAV'].iloc[SPLIT_DATE_INVESTOR:] = companies[7]['ADJ NAV'].iloc[SPLIT_DATE_INVESTOR:] * SPLIT_FACTOR_INVESTOR






In [12]:
#high DSSP and HIDI is good

AVG_DISC = pd.DataFrame(latour['PRIS'].copy())
AVG_DISC['PRIS'] = 0
AVG_DISC.columns = ['AVG_DISC']

for comp in companies:

    AVG_DISC['AVG_DISC'] = AVG_DISC['AVG_DISC'] + comp['P/D']

AVG_DISC = AVG_DISC/len(companies)

for comp in companies:
    comp['SPREAD'] = comp['P/D'] - AVG_DISC['AVG_DISC']
    #comp['DSSP'] = -comp['SPREAD'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['S3ZS'] = -comp['SPREAD'].rolling(look_back).apply(lambda x: mod_Z_score(x),raw=False) 
    #comp['HIDI'] = -comp['R/P'].rolling(look_back).apply(lambda x: calc_ecdf(x),raw=False) + 1
    comp['C3ZS'] = -comp['P/D'].rolling(look_back).apply(lambda x: mod_Z_score(x),raw=False)
    ### USe adjusted Pris and NAV
    comp['MOM'] = (comp['ADJ PRIS'].pct_change(252) + comp['ADJ PRIS'].pct_change(126) + comp['ADJ PRIS'].pct_change(63))/3
    comp['NAV MOM'] = (comp['ADJ NAV'].pct_change(252) + comp['ADJ NAV'].pct_change(126) + comp['ADJ NAV'].pct_change(63))/3
    comp['ST'] = comp['ADJ PRIS'].pct_change(ST_look_back)
    
    

In [13]:
INV_COMP = None

for comp in companies:
    
    INV_COMP = pd.concat([INV_COMP, comp.tail(1)], axis = 0)

INV_COMP['Company'] = companies_names
INV_COMP.set_index(['Company'], inplace=True, drop=True)

INV_COMP['P/D']=-INV_COMP['P/D']

In [14]:
INV_COMP['VRANK'] = (INV_COMP['P/D'].rank(ascending = False) + INV_COMP['S3ZS'].rank(ascending = False) +
 INV_COMP['C3ZS'].rank(ascending = False)).rank()



INV_COMP['MRANK'] = (INV_COMP['MOM'].rank(ascending = False)+INV_COMP['NAV MOM'].rank(ascending = False)).rank()



###### NOTE NOT EQUAL WEIGHT SCORES #############

INV_COMP['CRANK'] = (0.6*INV_COMP['VRANK'] + 0.4*INV_COMP['MRANK']).rank()


#################


INV_COMP['SIGNAL'] = INV_COMP.apply(lambda x: signal(x['CRANK']), axis=1)



weight_factor = 0.5

#INV_COMP['TARGET'] = INV_COMP['DEFWEIGHT']+INV_COMP['DEFWEIGHT']*INV_COMP['SIGNAL']*weight_factor

compact = ['P/D', 'C3ZS', 'S3ZS', 'MOM', 'NAV MOM', 'VRANK', 'MRANK', 'CRANK', 'ST', 'SIGNAL']

INV_COMP = INV_COMP[compact]

In [15]:
average = pd.DataFrame(INV_COMP.mean(axis = 0)).transpose()
average.index = ['Average'] 
INV_COMP = pd.concat([INV_COMP, average], axis = 0)

## Display Results

In [16]:
data_date  = re.match(r'\d{4}-\d{2}-\d{2}',f)

th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(INV_COMP.columns), as_cmap=True)

INV_COMP_VIEW = (INV_COMP.style
    .applymap(color_negative_red, subset=['P/D', 'MOM', 'NAV MOM'])           
    .background_gradient(axis = 0 ,subset = ['S3ZS', 'C3ZS'],cmap=cm_high_good)
    .background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','CRANK', 'ST'],cmap=cm_low_good)
    #.set_caption('Performance Metrics')
    .format({'P/D': "{:.1%}", 'S3ZS': "{:.2f}", 'C3ZS': "{:.2f}", 'MOM':"{:.1%}", 'NAV MOM': "{:.1%}",
            'Sortino': "{:.2f}", 'MAR': "{:.2f}", 'VRANK': "{:.0f}", 'MRANK': "{:.0f}", 'CRANK': "{:.0f}",
             'ST':"{:.1%}",'SIGNAL': "{:.1f}"})
    .set_caption("Investmentbolag Rank {}".format(data_date.group(0)))
    .set_table_styles(styles)
    .set_properties(**{'color': 'black'}, subset = ['C3ZS','S3ZS','VRANK', 'MRANK','CRANK', 'ST']) 
    )

In [17]:
INV_COMP_VIEW

,P/D,C3ZS,S3ZS,MOM,NAV MOM,VRANK,MRANK,CRANK,ST,SIGNAL
Traction,4.9%,0.08,-0.53,3.0%,1.9%,8,1,5,2.0%,0.0
Lundbergsföretagen,7.6%,0.85,0.13,-7.6%,-0.6%,3,2,1,1.4%,1.0
Öresund,-1.0%,0.13,-0.52,-16.2%,-16.8%,6,6,6,3.5%,0.0
Industrivärden,13.3%,0.57,-0.48,-15.0%,-9.0%,4,4,3,0.9%,0.5
Svolder,-3.7%,-0.24,-1.74,-54.8%,-51.6%,10,10,11,9.5%,-1.0
Bure,-7.3%,0.13,-0.39,-37.7%,-22.4%,7,9,9,3.7%,-0.5
Creades,-14.5%,0.34,0.15,-30.8%,-9.8%,5,8,7,2.1%,0.0
Investor,17.3%,0.01,-0.86,-12.2%,-5.2%,8,3,8,0.9%,-0.5
Kinnevik,26.2%,0.93,0.95,-25.9%,-4.8%,2,5,2,0.8%,1.0
VNV Global,60.9%,2.21,2.16,-66.2%,-27.1%,1,10,4,-5.7%,0.5


In [18]:
import dataframe_image as dfi
exp_file_name = "results/inv_comp_{}".format(data_date.group(0)) + ".png"
dfi.export(INV_COMP_VIEW, exp_file_name)